In [ ]:
import pandas as pd
import numpy as np
import os
import nltk

In [ ]:
os.environ['KAGGLE_CONFIG_DIR']= '/content'

In [ ]:
import the dataset using kaggle api
!kaggle datasets download -d alanvourch/tmdb-movies-daily-updates

In [ ]:
!unzip /content/tmdb-movies-daily-updates.zip

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/data set/TMDB_all_movies.csv") #main data frame

In [ ]:
df.columns

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'budget', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'cast', 'director', 'director_of_photography', 'writers', 'producers',
       'music_composer', 'imdb_rating', 'imdb_votes'],
      dtype='object')

In [ ]:
df['release_date'] = pd.to_datetime(df['release_date'])

In [ ]:
df['release_date'].min()

Timestamp('1800-01-01 00:00:00')

In [ ]:
df=df[df['release_date'] > '1999-12-31']

In [ ]:
s=df[['original_language','id']].groupby('original_language').count().sort_values(by='id',ascending =False).iloc[:15]

In [ ]:
s.index

Index(['en', 'fr', 'es', 'ja', 'de', 'pt', 'zh', 'ru', 'it', 'ko', 'nl', 'sv',
       'hi', 'ar', 'tl'],
      dtype='object', name='original_language')

In [ ]:
k=['en', 'fr', 'es', 'de', 'ja', 'pt', 'zh', 'ru', 'it','or','ko', 'ar', 'cs',
       'nl', 'sv', 'hi']

In [ ]:
df=df[df['original_language'].isin(k)]

In [ ]:
df.head(1)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,production_countries,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes
4,8,Life in Loops (A Megacities RMX),7.5,27.0,Released,2006-01-01,0.0,80.0,42000.0,tt0825671,...,Austria,"English, हिन्दी, 日本語, Pусский, Español",NaN,Timo Novotny,Wolfgang Thaler,"Michael Glawogger, Timo Novotny","Ulrich Gehmacher, Timo Novotny",NaN,8.1,282.0


In [ ]:
#let discuse which columns we want for our model
#1. 'id' 2. 'genres' 3. 'taglines 4. 'title' (not original title becuse movie name can vary based uopon region)
#5. 'overview' 6.'production_companies' 7.'cast' 8. 'crew'

In [ ]:
movies = df[['id','original_language','genres','title','overview','cast','director']]

In [ ]:
#we need only three columns for our apps (movied_id,titile,tags )

In [ ]:
movies.isnull().sum()

,0
id,0
original_language,0
genres,132288
title,3
overview,76932
cast,165512
director,88604


In [ ]:
#!kaggle datasets download -d asaniczka/tmdb-movies-dataset-2023-930k-movies

In [ ]:
#!unzip /content/tmdb-movies-dataset-2023-930k-movies.zip

In [ ]:
s=pd.read_csv('/content/drive/MyDrive/data set/TMDB_movie_dataset_v11.csv') #this data set is for supporting th above data (from this data i will take the key word)

In [ ]:
s.columns

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords'],
      dtype='object')

In [ ]:
keywords=s[['id','keywords']]

In [ ]:
movies=pd.merge(movies,keywords,on ='id',how='inner')

In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499352 entries, 0 to 499351
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   id                 499352 non-null  int64 
 1   original_language  499352 non-null  object
 2   genres             367046 non-null  object
 3   title              499349 non-null  object
 4   overview           422437 non-null  object
 5   cast               333833 non-null  object
 6   director           410783 non-null  object
 7   keywords           142160 non-null  object
dtypes: int64(1), object(7)
memory usage: 30.5+ MB


In [ ]:
movies.isnull().sum()

,0
id,0
original_language,0
genres,132306
title,3
overview,76915
cast,165519
director,88569
keywords,357192


In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.isnull().sum()

,0
id,0
original_language,0
genres,0
title,0
overview,0
cast,0
director,0
keywords,0


In [ ]:
movies[movies['title']=='Avatar']

,id,original_language,genres,title,overview,cast,director,keywords
5795,19995,en,"Action, Adventure, Fantasy, Science Fiction",Avatar,"In the 22nd century, a paraplegic Marine is di...","Laz Alonso, Sean Anthony Moran, Kelly Kilgour,...",James Cameron,"future, society, culture clash, space travel, ..."


In [ ]:
movies=movies[~(movies['id'].duplicated())]

In [ ]:
movies[movies['id'].duplicated()]

,id,original_language,genres,title,overview,cast,director,keywords


In [ ]:
movies=movies[~(movies['title'].duplicated())]

In [ ]:
movies[movies['title'].duplicated()]

,id,original_language,genres,title,overview,cast,director,keywords


In [ ]:
len(movies['original_language'].value_counts())

16

In [ ]:
movies.shape

(89951, 8)

In [ ]:
movies.columns

Index(['id', 'original_language', 'genres', 'title', 'overview', 'cast',
       'director', 'keywords'],
      dtype='object')

clean data movies (147979 data)

In [ ]:
movies.head(5)

,id,original_language,genres,title,overview,cast,director,keywords
2,12,en,"Animation, Family",Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...","Bob Bergen, LuLu Ebeling, Bob Peterson, Caroli...",Andrew Stanton,"sydney, australia, parent child relationship, ..."
3,16,en,"Drama, Crime",Dancer in the Dark,"Selma, a Czech immigrant on the verge of blind...","Eric Voge, Anders Thorhauge, Anthony Ajoise Ol...",Lars von Trier,"factory worker, dying and death, individual, i..."
4,17,en,"Horror, Thriller, Mystery",The Dark,"In an attempt to pull her family together, Adè...","Maurice Roëves, Tonya Smith, Richard Elfyn, Se...",John Fawcett,"sea, wales, child abuse, shepherd, adolescence..."
5,20,en,"Drama, Romance",My Life Without Me,A fatally ill mother with only two months to l...,"Jessica Amlee, Alfred Molina, Mark Ruffalo, Am...",Isabel Coixet,"dying and death, daughter, farewell, night shi..."
6,22,en,"Adventure, Fantasy, Action",Pirates of the Caribbean: The Curse of the Bla...,"Jack Sparrow, a freewheeling 18th-century pira...","James McAuley, Michael Sean Tighe, Israel Adur...",Gore Verbinski,"blacksmith, east india company, gold, exotic i..."


In [ ]:
movies['cast'].head(3)

,cast
2,"Bob Bergen, LuLu Ebeling, Bob Peterson, Caroline Kindred, Jeff Pidgeon, Stephen Root, Bobby Block, Albert Brooks, Brad Garrett, Austin Pendleton, Erica Beck, Alec Medlock, Lee Unkrich, Nicholas Bird, Bradley Trevor Greive, Leland Grossman, Allison Janney, John Ratzenberger, Rove McManus, Susan Blu, Katherine Ringgold, Ellen DeGeneres, Lisa Peers, Laraine Newman, Danny Mann, Mickie McGowan, Noah Luke, Kali Whitehurst, Eliza Schneider, Joe Ranft, Emmett Shoemaker, Jane Carr, Nicole Miller, Bruce Spence, Jan Rabson, Barry Humphries, Eric Bana, Phil Proctor, Michaela Jill Murphy, Sherry Lynn, Carlos Alazraqui, Andrew Stanton, Lili Ishida, Jess Harnell, Marc John Jefferies, David Ian Salter, Vanessa Marano, Erik Per Sullivan, Alexander Gould, Willem Dafoe, Paul Eiding, Annelise Nolting, Aaron Fors, Jack Angel, Elizabeth Perkins, Laura Marano, Geoff Brooks, Daryl Sabara, James S. Baker, Vicki Lewis, Jim Ward, Ben Stanton, Jordan Ranft, Evan Sabara, Bill Hunter, Oliver Kindred, Geoffrey Rush, Jennifer Darling"
3,"Eric Voge, Anders Thorhauge, Anthony Ajoise Olufemi Jacob, Katrine Falkenberg, Bjorn Ahlander, Noah Lazarus, Anders Tärneberg, Andrew Lucre, Ana Christine Broström, Jens Albinus, Rikke Lylloff, Luke Reilly, T.J. Rizzo, Birgitte Skands, Karoliina Heiskanen, Caroline Sascha Cogez, Cristian Valle, Udo Kier, Ed Hickok, Paprika Steen, Vladica Kostic, Sheldon Litt, Troels Asmussen, Björk, Anna Rosenberg, Annette Lindholm, Joel Grey, Ami Eklöf-Annell, Anna Norberg, Alex Mouro, Allan Gyldenkærne, Erik Drugge, Edvin Karsson, Timm Zimmermann, Reathel Bean, Stellan Skarsgård, Marianne Bengtsson, Cara Seymour, Siobhan Fallon Hogan, Nick Wolf, Al Agami, Alexander Arli, Anders-Peter Torsleff Hansen, Diana G.L. Watson, Mette Berggreen, Sean-Michael Smith, Bo Westerholm, John Martinus, Frederik Mondrup, Zeljko Ivanek, Peter Stormare, Britt Bendixen, Erik Dammann, Lars Michael Dinesen, Carl Johan De Neergaard, Ann Crosset, David Morse, John Randolph Jones, Catherine Deneuve, Carol Linda Nielsen, Emilie Bendz, Fredrik Börgesson, Jean-Marc Barr, Anna David, Elin Johansson, Eli Stalhand, Bobo Eriksson, Vincent Paterson, Michael Flessas, Claus Berenhard"
4,"Maurice Roëves, Tonya Smith, Richard Elfyn, Sean Bean, Robin Griffith, Maria Bello, Sophie Stuckey, Abigail Stone, Casper Harvey, Gwenyth Petty, Eluned Jones"


**bold text** make(genres,overview,cast,director,keywords) columns to similar for furthur analysis


In [ ]:
#we want only 3-4 actor name
movies['cast'] = movies['cast'].str.split(',').apply(lambda x: [i.strip() for i in x[:4]])  # this code will take only 4 actor name and trim the unnesesary space
  # this code willremove the inner space "sam williongton"---> "samwilliongton" . which help our model

In [ ]:

movies['cast']=movies['cast'].apply(lambda x: [i.replace(" ","") for i in x])

In [ ]:
movies['cast'][2]

['BobBergen', 'LuLuEbeling', 'BobPeterson', 'CarolineKindred']

In [ ]:
#similarly we want to convert for the director
movies['director']=movies['director'].str.split(',').apply(lambda x: [i.strip() for i in x] )

movies['director']=movies['director'].apply(lambda x: [i.replace(" ","") for i in x])

In [ ]:
movies['director'][100]

['DeepaMehta']

In [ ]:
#similarly for the genre columns
movies['genres']=movies['genres'].str.split(',').apply(lambda x : [i.strip() for i in x])

movies['genres']=movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])

In [ ]:
movies['genres'][2]

['Animation', 'Family']

In [ ]:
#similarly for the key words
movies['keywords']=movies['keywords'].str.split(',').apply(lambda x : [i.strip() for i in x])

movies['keywords']=movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])

In [ ]:
movies['keywords'][6]

['blacksmith',
 'eastindiacompany',
 'gold',
 'exoticisland',
 'governor',
 'skeleton',
 'jamaica',
 'britishempire',
 'pirate',
 'swashbuckler',
 '18thcentury',
 'caribbeansea',
 'aftercreditsstinger',
 'pirateship',
 'britishnavy',
 'tortuga',
 'basedonthemeparkride']

In [ ]:
#similarly for the over view
movies['overview']=movies['overview'].str.split().apply(lambda x : [i.strip() for i in x])

movies['overview']=movies['overview'].apply(lambda x: [i.replace(" ","") for i in x])

Create a new columns tags combining the column (genres,title,overview,cast,director,keywords) for our model

In [ ]:
movies['tags']=movies['overview']+ movies['genres'] + movies['keywords'] + movies['cast'] + movies['director']

In [ ]:
movies.reset_index(inplace=True)

In [ ]:
movies.head()

,index,id,original_language,genres,title,overview,cast,director,keywords,tags
0,2,12,en,"[Animation, Family]",Finding Nemo,"[Nemo,, an, adventurous, young, clownfish,, is...","[BobBergen, LuLuEbeling, BobPeterson, Caroline...",[AndrewStanton],"[sydney, australia, parentchildrelationship, a...","[Nemo,, an, adventurous, young, clownfish,, is..."
1,3,16,en,"[Drama, Crime]",Dancer in the Dark,"[Selma,, a, Czech, immigrant, on, the, verge, ...","[EricVoge, AndersThorhauge, AnthonyAjoiseOlufe...",[LarsvonTrier],"[factoryworker, dyinganddeath, individual, imm...","[Selma,, a, Czech, immigrant, on, the, verge, ..."
2,4,17,en,"[Horror, Thriller, Mystery]",The Dark,"[In, an, attempt, to, pull, her, family, toget...","[MauriceRoëves, TonyaSmith, RichardElfyn, Sean...",[JohnFawcett],"[sea, wales, childabuse, shepherd, adolescence...","[In, an, attempt, to, pull, her, family, toget..."
3,5,20,en,"[Drama, Romance]",My Life Without Me,"[A, fatally, ill, mother, with, only, two, mon...","[JessicaAmlee, AlfredMolina, MarkRuffalo, Aman...",[IsabelCoixet],"[dyinganddeath, daughter, farewell, nightshift...","[A, fatally, ill, mother, with, only, two, mon..."
4,6,22,en,"[Adventure, Fantasy, Action]",Pirates of the Caribbean: The Curse of the Bla...,"[Jack, Sparrow,, a, freewheeling, 18th-century...","[JamesMcAuley, MichaelSeanTighe, IsraelAduramo...",[GoreVerbinski],"[blacksmith, eastindiacompany, gold, exoticisl...","[Jack, Sparrow,, a, freewheeling, 18th-century..."


we need id,title,tags columns from the above table

In [ ]:
movies.columns

Index(['index', 'id', 'original_language', 'genres', 'title', 'overview',
       'cast', 'director', 'keywords', 'tags'],
      dtype='object')

In [ ]:
movies.shape

(89951, 10)

new_df is created


In [ ]:
new_df=movies[['id','original_language','title','tags']]

In [ ]:
#converting all the list tags to string
new_df['tags']=new_df['tags'].apply(lambda x : " ".join(x))

<ipython-input-213-4fbcf56c62ac>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x : " ".join(x))


In [ ]:
#converting all the tags to lowercase
new_df['tags']=new_df['tags'].apply(lambda x: x.lower())

<ipython-input-214-1f515a259427>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x: x.lower())


In [ ]:
new_df.head()

,id,original_language,title,tags
0,12,en,Finding Nemo,"nemo, an adventurous young clownfish, is unexp..."
1,16,en,Dancer in the Dark,"selma, a czech immigrant on the verge of blind..."
2,17,en,The Dark,"in an attempt to pull her family together, adè..."
3,20,en,My Life Without Me,a fatally ill mother with only two months to l...
4,22,en,Pirates of the Caribbean: The Curse of the Bla...,"jack sparrow, a freewheeling 18th-century pira..."


In [ ]:
#natural language processing which will ( [loved, loving ,loves]--->[love,love,love])
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()

In [ ]:
#helper function
def stem (text):
  y=[]

  for i in text.split():
    y.append(ps.stem(i))

  return " ".join(y)

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)

<ipython-input-218-402dd7cb6b10>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [ ]:
new_df.shape

(89951, 4)

In [ ]:
new_df=new_df[new_df['original_language']=='en']

In [ ]:
new_df.to_csv("new_df.csv",index=False)

Rest in part2
